In [88]:
import numpy as np
import pandas as pd
import os
import importlib

In [89]:
import ipynb.fs.full.wspMultithreshold as wspMultithreshold
import ipynb.fs.full.npyHandler as npyHandler
import ipynb.fs.full.wspTsallisEvaluation as wspTsallisEvaluation

importlib.reload(wspMultithreshold)
importlib.reload(npyHandler)
importlib.reload(wspTsallisEvaluation)

<module 'ipynb.fs.full.wspTsallisEvaluation' (/media/webis/Data Both/OneDrive - FEI/Projetos/GitHub/image-segmentation/wspTsallisEvaluation.ipynb)>

### Automatic q Calculation

$$S_{MAX} = \frac{1}{q-1} [1 - n(p^q(x))]$$

In [90]:
q_value = np.linspace(0.01, 1.99, num=199)
#q_value = np.linspace(0.001, 1.999, num=1999)
#q_value = np.linspace(0.0001, 1.9999, num=19999)

q_value[q_value == 1.00] = 1.00000000000001

In [95]:
Sq_Smax_value = []

In [96]:
def get_best_q(image_path):
    global Sq_Smax_value
    pixel_array = npyHandler.read_npy_image(image_path)
    hist, _, _, _ = wspMultithreshold.wspGrayHistogram(pixel_array)

    lb, ub = pixel_array.min(), pixel_array.max()

    N = ub - lb + 1
    p_x = 1/N

    Sq_Smax_value = []

    for q in q_value:
        Sq = wspTsallisEvaluation.TsallisEntropy(hist, q, 0, N)
        
        Smax = 1/(q-1) * (1-(N*((p_x)**q)))
        
        Sq_Smax = Sq / Smax
        Sq_Smax_value.append(Sq_Smax)


    sorted_indices = np.argsort(Sq_Smax_value)
    second_lowest_index = sorted_indices[0]

    return q_value[second_lowest_index]
        


In [94]:
def plot_q_curve():

    import matplotlib.pyplot as plt

    x = q_value
    y = Sq_Smax_value

    #lowest_index = np.argmin(y)

    sorted_indices = np.argsort(y)
    second_lowest_index = sorted_indices[0]

    # Plotting the line graph
    plt.plot(x, y)

    #plt.scatter(x[lowest_index], y[lowest_index], color='red', label='Lowest Point')
    annotation_text = f'(q = {x[second_lowest_index]: .2f})'
    plt.annotate(annotation_text, xy=(x[second_lowest_index], y[second_lowest_index]), xytext=(x[second_lowest_index], y[second_lowest_index]+0.2), arrowprops=dict(arrowstyle='->'))

    # Adding labels and title
    plt.xlabel('q')
    plt.ylabel('Sq / Smax')

    # Display the plot
    plt.savefig("Automatic q value.jpg", bbox_inches='tight')
    plt.show()